# Precalculate FLORIS Solutions

Use FLASC tools to precalculate the FLORIS solutions across wind speeds and directions without wake steering to enable faster comparisons in later steps. The FLORIS solutions based on four different wake models are then saved in the "/precalculated_floris_solutions" folder. 

In [4]:
from pathlib import Path
from time import perf_counter as timerpc

import numpy as np
from floris.tools import ParallelComputingInterface

from flasc.utilities import floris_tools as ftools
from flasc.utilities.utilities_examples import load_floris_smarteole as load_floris

In [5]:
# User settings
max_workers = 16
wake_models = ["jensen", "turbopark", "gch", "cc"]

In [6]:
# Precalculate FLORIS solutions
root_path = Path.cwd() / "precalculated_floris_solutions"
root_path.mkdir(exist_ok=True)

for wake_model in wake_models:
    fn = root_path / "df_fi_approx_{:s}.ftr".format(wake_model)
    if fn.is_file():
        print("FLORIS table for '{:s}' model exists. Skipping...".format(wake_model))
        continue

    start_time = timerpc()
    print("Precalculating FLORIS table for '{:s}' model...".format(wake_model))
    fi, _ = load_floris(wake_model=wake_model)
    fi_pci = ParallelComputingInterface(
        fi=fi,
        max_workers=max_workers,
        n_wind_direction_splits=max_workers,
        print_timings=True,
    )
    df_fi_approx = ftools.calc_floris_approx_table(
        fi=fi_pci,
        wd_array=np.arange(0.0, 360.01, 3.0),
        ws_array=np.arange(1.0, 30.01, 1.0),
        ti_array=[0.03, 0.06, 0.09, 0.12, 0.15],
    )
    end_time = timerpc()
    print("Computation time: {:.2f} s".format(end_time - start_time))
    df_fi_approx.to_feather(fn)

Precalculating FLORIS table for 'jensen' model...
Generating a df_approx table of FLORIS solutions covering a total of 18150 cases.
Total time spent for parallel calculation (16 workers): 6.492 s
  Time spent in parallel preprocessing: 0.005 s
  Time spent in parallel loop execution: 6.486 s.
  Time spent in parallel postprocessing: 0.000 s
Total time spent for parallel calculation (16 workers): 6.612 s
  Time spent in parallel preprocessing: 0.005 s
  Time spent in parallel loop execution: 6.607 s.
  Time spent in parallel postprocessing: 0.000 s
Total time spent for parallel calculation (16 workers): 7.267 s
  Time spent in parallel preprocessing: 0.005 s
  Time spent in parallel loop execution: 7.261 s.
  Time spent in parallel postprocessing: 0.000 s
Total time spent for parallel calculation (16 workers): 7.577 s
  Time spent in parallel preprocessing: 0.005 s
  Time spent in parallel loop execution: 7.572 s.
  Time spent in parallel postprocessing: 0.000 s
Total time spent for par